In [ ]:
import requests
import pandas as pd
import numpy as np
import nbconvert
import datetime
import plotly.graph_objects as go
import warnings; warnings.simplefilter('ignore')
from IPython.display import display, Markdown, Image, SVG
import IPython.display as display
import re
import bmondata
from bmondata import Server
from dateutil.relativedelta import relativedelta
import scrapbook as sb

In [ ]:
# Parameters to be changed/exported using Papermill or Scrapbook
building_id = 9
server_web_address = 'https://bms.ahfc.us'

In [ ]:
# 'Glue' down variables for scrapbook to export later
sb.glue('sort_order', 2)
sb.glue('title', 'Fuel Use')

server = Server(server_web_address)
building_df = server.buildings(building_id)
current_building_name = building_df[0]['title']

fuel_ids = building_df[0]['fuel_ids']
outdoor_temps = building_df[0]['outdoor_temp']
current_building_type = building_df[0]['building_type']

if len(fuel_ids) == 0:
    error_message = 'This building does not appear to have any fuel data. Either there is no data source or it has not been properly configured.'
    raise RuntimeError(error_message)

all_buildings = server.buildings()
org_df = pd.DataFrame(all_buildings)

title_md = '''# Fuel Consumption for {} building'''
title_md = title_md.format(current_building_name)

##############################################################


Markdown(title_md)

## Climate-Normalized Daily Consumption

In [ ]:
sensors = [ 
(fuel_ids, 'fuel_usage'),
(outdoor_temps, 'outdoor_temp')
]

fuel_averages = server.sensor_readings(sensors,
                                       start_ts = datetime.datetime.now() - relativedelta(years=5),
                                       end_ts = datetime.datetime.now(),
                                       averaging = '1H')

fuel_averages = fuel_averages.reset_index()

fuel_averages = fuel_averages.rename(columns={'index':'datetime_col'})

fuel_averages['date'] = fuel_averages.datetime_col.apply(lambda x: x.date())

fuel_averages['time'] = fuel_averages.datetime_col.apply(lambda x: x.time())

fuel_averages['year'] = fuel_averages.datetime_col.apply(lambda x: x.year)
fuel_averages['month'] = fuel_averages.datetime_col.apply(lambda x: x.month)
fuel_averages['day'] = fuel_averages.datetime_col.apply(lambda x: x.day)

# Base 60 heating degree hours
fuel_averages['heating_degree_days'] = (60.0 - fuel_averages.outdoor_temp) / 24

# Get rid of negative values for fuel averages as these are likely erroneous data
fuel_averages = fuel_averages.query("fuel_usage >= 0")

daily_fuel_use = fuel_averages.groupby(['year', 'month', 'day']).sum()

daily_fuel_use['normalized_fuel_usage'] = daily_fuel_use.fuel_usage / daily_fuel_use.heating_degree_days

daily_fuel_use = daily_fuel_use.reset_index()
daily_fuel_use['date'] = pd.to_datetime((dict(year=daily_fuel_use.year, month=daily_fuel_use.month, day=daily_fuel_use.day)))

daily_fuel_use['dayofweek'] = daily_fuel_use.date.apply(lambda x: x.dayofweek)

current_month_start = daily_fuel_use.date.iloc[-1] - pd.offsets.Day(28)

current_month = daily_fuel_use.query("date >= @current_month_start")

historical_complete = daily_fuel_use.query("date < @current_month_start")

month_day_avgs = historical_complete.groupby(['month', 'dayofweek']).mean()
month_day_avgs = month_day_avgs.reset_index()

current_month_w_day_avgs = pd.merge(current_month, month_day_avgs, how='left',
                                  left_on=['month', 'dayofweek'],
                                  right_on=['month', 'dayofweek'], 
                                  suffixes=('', '_historical_avg'))

current_month_w_day_avgs['current_use_vs_historical_difference'] = current_month_w_day_avgs.normalized_fuel_usage - current_month_w_day_avgs.normalized_fuel_usage_historical_avg

# Define anomalous range as three standard deviations above the mean
three_std = current_month_w_day_avgs.current_use_vs_historical_difference.mean() + current_month_w_day_avgs.current_use_vs_historical_difference.std() * 3

# This works, but the graph looks odd unless we get one data point on either side to connect them
current_month_w_day_avgs['fuel_use_anomalies'] = np.where(current_month_w_day_avgs.current_use_vs_historical_difference > three_std,
                                                            current_month_w_day_avgs.normalized_fuel_usage,
                                                            np.nan)

anomaly_df = current_month_w_day_avgs.query("fuel_use_anomalies == fuel_use_anomalies")

diff_series = np.diff(anomaly_df.index, n=1)

diff_series = np.insert(diff_series, 0, 0)
anomaly_df['diff_series'] = diff_series

# This keeps track of the number of groups of distinct anomalous periods in the data
group_counter = 0

# This padding is a cut-off used to group together time series indices that are close 
# enough to still be considered a group (e.g. there might be an anomaly at one time,
# followed by another anomalous reading 45 minutes later, which should probably all just
# be considered the same group)
index_padding = 4

for idx, row in anomaly_df.iterrows():
    if row['diff_series'] <= index_padding:
        anomaly_df.at[idx, 'group_number'] = group_counter
    else:
        group_counter += 1
        anomaly_df.at[idx, 'group_number'] = group_counter

anomaly_max = anomaly_df.groupby(['group_number']).max()[['date', 'normalized_fuel_usage']]
anomaly_min = anomaly_df.groupby(['group_number']).min()[['date', 'normalized_fuel_usage']]

anomaly_groups = pd.merge(anomaly_max, anomaly_min, how='outer',
                         left_index=True, right_index=True, 
                         suffixes=('_max', '_min'))

if anomaly_groups.date_min.isna().all():
    normalized_anomaly_rectangles = []
    image_list = []
else:
    normalized_anomaly_rectangles = []
    image_list = []
    for idx, row in anomaly_groups.iterrows():
        
        anomaly_start = row['date_min']
        anomaly_end = row['date_max']
        anomaly_midpoint = ((anomaly_end - anomaly_start) / 2) + anomaly_start
        
        normalized_anomaly_rectangles.append({'type':'rect',
                            'xref':'x',
                            'yref':'paper',
                            'x0':anomaly_start,
                            'y0':0,
                            'x1':anomaly_end,
                            'y1':1,
                            'fillcolor':('rgb(205, 12, 24)'),
                            'opacity':0.5,
                            'line': {
                                'width':1,
                            }
                            })
        
        image_list.append({'source': 'https://github.com/dustin-cchrc/cchrc_python_for_non_programmers/blob/master/energy_savings_icon.png',
                'xref': "paper",
                'yref': "paper",
                'x': 1.0, #anomaly_midpoint ,
                'y': 0.9,
                'sizex': 0.2,
                'sizey': 0.2,
                'xanchor': "right",
                'yanchor': "bottom"})

this_month = go.Scatter(x = current_month_w_day_avgs.date,
                   y = current_month_w_day_avgs.normalized_fuel_usage,
                   line = dict(color = ('rgb(22, 96, 167)')),
                   name='Current Month Normalized Fuel Consumption')

historical_avg = go.Scatter(x = current_month_w_day_avgs.date,
                   y = current_month_w_day_avgs.normalized_fuel_usage_historical_avg,
                   line = dict(dash = 'dashdot',
                              color = ('rgb(22, 96, 167)')),
                            opacity = 0.6,
                   name='Historical Average Normalized Fuel Consumption')

layout = dict(title = 'Current Normalized Fuel Consumption vs. Historical Average',
              xaxis = dict(title='Date and Time'),
              yaxis = dict(title='Daily Climate Normalized Fuel Consumption (BTUs / heating degree day)'),
              images = image_list,
              shapes = normalized_anomaly_rectangles
             )

data = [this_month, historical_avg]

fig = go.Figure(dict(data=data, layout=layout))

###########################################################################

fig.show()



if anomaly_groups.date_min.isna().all():
    md_results = '''#### There were no periods of extreme fuel consumption this week-- keep up the good work!'''
else:
     md_results = '''#### The periods highlighted in red signify much higher fuel consumption than normal after climate normalization. Try to identify what happened and how to prevent it in the future.'''

        
        
Markdown(md_results)

In [ ]:
import markdown
md = markdown.Markdown()

md_for_html = '''## Current Normalized Fuel Consumption
  - **Description:** This graph shows the fuel consumption of the building over the past month as compared to historical data. The solid line shows the daily fuel consumption in BTUs divided by the actual measured base 60 heating degree days for the past month. The dashed line shows the historical climate normalized data for the same month (e.g. all data from Octobers in previous years) averaged by the day of the week (to account for occupied vs. unoccupied periods). 
  - **Anomalies:** Points of high energy use are highlighted in red-- these are times when the recent month's climate-normalized fuel usage were three standard deviations above the mean for that day.
  - **Potential for savings:** This data can help users identify problems in heating, hot water, and controls systems by flagging excessive fuel usage.
'''

converted_html = md.convert(md_for_html)

click_html = '<details><summary>Click for details</summary>'
end_click = '</details>'
final_converted_html = click_html + converted_html + end_click



display.HTML(final_converted_html)

## Fuel Use Last Week

In [ ]:
current_week_start = fuel_averages.datetime_col.iloc[-1] - pd.offsets.Day(7)
current_week = fuel_averages.query("datetime_col >= @current_week_start")

last_week_start = current_week_start - pd.offsets.Day(7)
last_week_df = fuel_averages.query("datetime_col >= @last_week_start")
last_week_df = last_week_df.query("datetime_col < @current_week_start")

two_weeks_ago_start = last_week_start - pd.offsets.Day(7)
two_weeks_ago_df = fuel_averages.query("datetime_col >= @two_weeks_ago_start")
two_weeks_ago_df = two_weeks_ago_df.query("datetime_col < @last_week_start")

three_weeks_ago_start = two_weeks_ago_start - pd.offsets.Day(7)
three_weeks_ago_df = fuel_averages.query("datetime_col >= @three_weeks_ago_start")
three_weeks_ago_df = three_weeks_ago_df.query("datetime_col < @two_weeks_ago_start")

# Adjust the datetime column so the visualization will have the electricity data overlaying the current week.
last_week_df['adjusted_datetime_col'] = last_week_df.datetime_col.apply(lambda x: x + pd.offsets.Day(7))
two_weeks_ago_df['adjusted_datetime_col'] = two_weeks_ago_df.datetime_col.apply(lambda x: x + pd.offsets.Day(14))
three_weeks_ago_df['adjusted_datetime_col'] = three_weeks_ago_df.datetime_col.apply(lambda x: x + pd.offsets.Day(21))

this_week = go.Scatter(x = current_week.datetime_col,
                   y = current_week.fuel_usage,
                   line = dict(color = '#ca0020'),
                   name='Current week fuel consumption')

last_week = go.Scatter(x = last_week_df.adjusted_datetime_col,
                      y = last_week_df.fuel_usage,
                       line = dict(dash = 'dashdot',
                                   color = '#3182bd'),
                        # opacity = 1.0,
                       name = "Fuel consumption last week")

two_weeks_ago = go.Scatter(x = two_weeks_ago_df.adjusted_datetime_col,
                          y = two_weeks_ago_df.fuel_usage,
                           line = dict(dash = 'dashdot',
                                       color = '#6baed6'),
                          #   opacity = 1.0,
                           name = "Fuel consumption 2 weeks ago")

three_weeks_ago = go.Scatter(x = three_weeks_ago_df.adjusted_datetime_col,
                            y = three_weeks_ago_df.fuel_usage,
                             line = dict(dash = 'dashdot',
                                         color = '#bdd7e7'),
                           #    opacity = 1.0,
                            name = "Fuel consumption 3 weeks ago")

layout = dict(title = 'Fuel Consumption: Current vs. Recent Weeks',
              xaxis = dict(title='Date and Time'),
              yaxis = dict(title='Fuel Consumption (BTUs/hour)')
             )

data = [this_week, last_week, two_weeks_ago, three_weeks_ago]

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [ ]:
new_markdown_for_html = '''## Fuel Consumption: Current vs. Recent Weeks
- **Description:** This graph shows the hourly fuel consumption for this building over the past week, as compared to the three weeks prior. The solid red line shows the hourly fuel consumption in BTUs, and the dashed blue lines show the hourly fuel consumption in previous weeks. This data has not been normalized by climate, so it will be affected by recent temperature fluctuations.
- **Potential for savings:** This data can help users identify problems in heating, hot water, and controls systems by flagging unexpected changes in fuel consumption over the past month.
'''

new_converted_html = md.convert(new_markdown_for_html)
second_converted_html = click_html + new_converted_html + end_click



display.HTML(second_converted_html)

## Unoccupied Fuel Consumption

In [ ]:
# Get schedule and timezone from the API data
building_schedule = building_df[0]['schedule']
building_timezone = building_df[0]['timezone']

# Create a schedule object using Ian's library
schedule_object = bmondata.Schedule(building_schedule, building_timezone)

if building_schedule is '' or schedule_object is None:
    md_results_2 = '''#### <font color='red'>There is no occcupied schedule entered for this building.</font>'''
else:
    # Set the start and end times of the graph
    graph_start_date = datetime.datetime.now() - relativedelta(weeks=1)
    graph_end_date = datetime.datetime.now()

    # Use the schedule object to create a list of tuples with the occupied start and end times falling within the graph range
    list_of_occupied_timestamps = schedule_object.occupied_periods(datetime.datetime.timestamp(graph_start_date), datetime.datetime.timestamp(graph_end_date))

    # Loop through the list of occupied timestamps and convert them to datetimes
    start_time_list = []
    end_time_list = []

    for i in np.arange(0, len(list_of_occupied_timestamps)):
        start_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][0])
        start_time_list.append(start_datetime)
        end_datetime = datetime.datetime.fromtimestamp(list_of_occupied_timestamps[i][1])
        end_time_list.append(end_datetime)

    # Create variables to indicate the first occupied start time and the last occupied end time
    occ_start = start_time_list[0]
    occ_end = end_time_list[-1]
    occ_day_list = np.arange(occ_start.weekday(), occ_end.weekday()+1)
    
    min_value = current_week.fuel_usage.min()
    max_value = current_week.fuel_usage.max()
    
    shape_list = []
    shape_dict = {'type':'rect',
                  'fillcolor':'#bdbdbd',
                  'opacity':0.35,
                  'line': {'width':1},
                  'layer':'below',
                  'y0':min_value,
                  'y1':max_value
                 }
    
    if graph_start_date < start_time_list[0]:
        shape_dict.update(x0=graph_start_date)
        shape_dict.update(x1=start_time_list[0])
        shape_list.append(shape_dict.copy())
        
    for i in np.arange(len(start_time_list)):
        if start_time_list[-1] != start_time_list[i]:
            shape_dict.update(x0=end_time_list[i])
            shape_dict.update(x1=start_time_list[i+1])
            shape_list.append(shape_dict.copy())

    if graph_end_date > end_time_list[-1]:
        shape_dict.update(x0=end_time_list[-1])
        shape_dict.update(x1=graph_end_date)
        shape_list.append(shape_dict.copy())
        
    fuel_usage = go.Scatter(x=current_week.datetime_col,
                             y=current_week.fuel_usage,
                             name="Fuel Consumption")

    layout = dict(title='Current Week Fuel Consumption',
              xaxis=dict(title='Date and Time'),
              yaxis=dict(title='Fuel Consumption (BTUs))'),
              shapes = shape_list
             )
    
    fig = go.Figure(dict(data=[fuel_usage], layout=layout))
    
    md_results_2 = ''
    
    fig.show()

    

Markdown(md_results_2)

## Grey areas indicate that the building is unoccupied; if fuel consumption does not decrease significantly during these periods, check to see if:
 - The thermostat is set back at night
 - Building ventilation is reduced at night
 - The building is being operated differently on weekends and holidays.

## Historical Fuel Use vs. Outdoor Temperature

In [ ]:
daily_fuel_averages = fuel_averages.groupby(['year', 'month', 'day']).mean()

daily_fuel_averages = daily_fuel_averages.reset_index()
daily_fuel_averages['datetime_col'] = pd.to_datetime((dict(year=daily_fuel_averages.year, month=daily_fuel_averages.month, day=daily_fuel_averages.day)))

one_year_start_date = daily_fuel_averages.datetime_col.iloc[-1] - relativedelta(years=1)

one_year_daily_fuel_averages = daily_fuel_averages.query("datetime_col >= @one_year_start_date")

one_year_daily_fuel_averages['timestamp'] = one_year_daily_fuel_averages.datetime_col.apply(lambda x: datetime.datetime.timestamp(x))

# Add a column with an indicator of whether the day is majority occupied or not
if schedule_object is None:
    one_year_daily_fuel_averages['occupied'] = 1
else: 
    one_year_daily_fuel_averages['occupied'] = one_year_daily_fuel_averages.timestamp.apply(lambda x: schedule_object.is_occupied(x, resolution='day'))

occupied_daily_fuel_averages = one_year_daily_fuel_averages.query("occupied == True")
unoccupied_daily_fuel_avearges = one_year_daily_fuel_averages.query("occupied == False")

seven_days_ago = datetime.datetime.now() - relativedelta(days=7)

last_seven_days_occupied = occupied_daily_fuel_averages.query("datetime_col >= @seven_days_ago")
before_seven_days_occupied = occupied_daily_fuel_averages.query("datetime_col < @seven_days_ago")
last_seven_days_unoccupied = unoccupied_daily_fuel_avearges.query("datetime_col >= @seven_days_ago")
before_seven_days_unoccupied = unoccupied_daily_fuel_avearges.query("datetime_col < @seven_days_ago")
today = one_year_daily_fuel_averages.iloc[-1]

##########################################################################

this_week_occupied = go.Scatter(x = last_seven_days_occupied.outdoor_temp,
                   y = last_seven_days_occupied.fuel_usage,
                                text = 'Last 7 Days',
                                type = 'scatter',
                                mode ='markers',
                                name = 'Last 7 Days',
                                marker = dict(color='#00CC00',
                                              symbol='circle',
                                              size=9)
                               )

this_week_unoccupied = go.Scatter(x = last_seven_days_unoccupied.outdoor_temp,
                   y = last_seven_days_unoccupied.fuel_usage,
                                text = 'Last 7 Days, Unoccupied',
                                type = 'scatter',
                                mode ='markers',
                                name = 'Last 7 Days, Unoccupied',
                                marker = dict(color='#00CC00',
                                              symbol='triangle-up',
                                              size=6)
                               )

before_seven_occupied = go.Scatter(x = before_seven_days_occupied.outdoor_temp,
                   y = before_seven_days_occupied.fuel_usage,
                                text =  '7+ Days Old',
                                type = 'scatter',
                                mode ='markers',
                                name = '7+ Days Old',
                                marker = dict(color='#2f7ed8',
                                              symbol='circle',
                                              size=9)
                               )

before_seven_unoccupied = go.Scatter(x = before_seven_days_unoccupied.outdoor_temp,
                   y = before_seven_days_unoccupied.fuel_usage,
                                text =  '7+ Days Old, Unoccupied',
                                type = 'scatter',
                                mode ='markers',
                                name = '7+ Days Old, Unoccupied',
                                marker = dict(color='#2f7ed8',
                                              symbol='triangle-up',
                                              size=6)
                               )

last_24_hours = go.Scatter(x = [today.outdoor_temp],
                   y = [today.fuel_usage],
                                text =  'Last 24 Hours',
                                type = 'scatter',
                                mode ='markers',
                                name = 'Last 24 Hours',
                                marker = dict(color='#FF0000',
                                              symbol='circle',
                                              size=9)
                               )

    
layout = dict(title = 'Fuel Consumption vs. Outdoor Temperature',
              xaxis = dict(title='Average Daily Temperature, (Deg F)'),
              yaxis = dict(title='Fuel Consumption (Daily Average BTUs/hour)'),
              height = 900
             )

data = [this_week_occupied, this_week_unoccupied, before_seven_occupied, before_seven_unoccupied, last_24_hours]

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

## Fuel Usage Compared to Similar Building Types

In [ ]:
markdown_error = ''

# Check to make sure there is a building type, if so, create visualization
if current_building_type is None:
    markdown_error = '''#### <font color='red'>This building has no building type currently associated with it.</font>'''
else:
    all_buildings_of_type = org_df.query("building_type == @current_building_type")
    all_buildings_of_type_list = []
    for fuel_id in all_buildings_of_type.fuel_ids.unique():
        if fuel_id != '':
            temp_df = server.sensor_readings((fuel_id, 'fuel_usage'),
                                                  start_ts = datetime.datetime.now() - relativedelta(years=3),
                                                    end_ts = datetime.datetime.now(),
                                                  averaging = '1H')
            temp_df['building_name'] = all_buildings_of_type.query("fuel_ids == @fuel_id").iloc[0]['title']
            temp_df['building_square_footage'] = all_buildings_of_type.query("fuel_ids == @fuel_id").iloc[0]['floor_area']
            all_buildings_of_type_list.append(temp_df)
    
    all_buildings_of_type_data = pd.concat(all_buildings_of_type_list)
    
    # Get rid of negative fuel numbers, as these don't make sense and are likely just erroneous data
    all_buildings_of_type_data = all_buildings_of_type_data.query("fuel_usage > 0")
    
    # Sum the BTUs (average BTUs/hour * hour) by month for each building
    monthly_grouped_data = all_buildings_of_type_data.groupby(['building_name', lambda x:x.year, lambda x: x.month]).sum()
    monthly_grouped_data = monthly_grouped_data.reset_index()
    monthly_grouped_data = monthly_grouped_data.rename(columns={'level_1':'year',
                                                               'level_2':'month',
                                                               'fuel_usage':'total_monthly_BTUs'})
    monthly_grouped_data = monthly_grouped_data.drop(columns='building_square_footage', axis=1)
    
    # Join the original building square footage data back to the dataset
    all_buildings_of_type_data = all_buildings_of_type_data[['building_name', 'building_square_footage']]
    all_buildings_of_type_data = all_buildings_of_type_data.drop_duplicates()
    all_buildings_of_type_data = pd.merge(monthly_grouped_data, all_buildings_of_type_data, how='left',
                                         left_on='building_name', right_on='building_name')
    

    # Normalize by square footage. Data starts as BTUs per month; result is in BTUs / square foot / month
    all_buildings_of_type_data['monthly_fuel_eui'] = all_buildings_of_type_data.total_monthly_BTUs / all_buildings_of_type_data.building_square_footage
    
    all_buildings_of_type_data['datetime'] = pd.to_datetime(dict(year=all_buildings_of_type_data.year, month=all_buildings_of_type_data.month, day=1))
    
    diverging_hues = ['#d73027','#fc8d59',
                  '#fee090','#ffffbf',
                  '#e0f3f8','#91bfdb',
                  '#4575b4']
    
    building_type_dict = {'OFFIC':'Office', 
                      'SCH':'School', 
                      'M-RES':'Multifamily Residential', 
                      'OTHER':'Miscellaneous Type'}
    
    current_building_type = building_type_dict[current_building_type]
    
    current_building_df = all_buildings_of_type_data.query("building_name == @current_building_name")
    all_buildings_avg_df = all_buildings_of_type_data.query("building_name != @current_building_name")
    all_buildings_avg_df = all_buildings_avg_df.groupby(['year', 'month']).mean()
    all_buildings_avg_df = all_buildings_avg_df.reset_index()
    all_buildings_avg_df['datetime'] = pd.to_datetime(dict(year=all_buildings_avg_df.year, month=all_buildings_avg_df.month, day=1))
    
    all_building_avg_monthly_fuel_eui = go.Scatter(x=all_buildings_avg_df.datetime,
                                                   y=all_buildings_avg_df.monthly_fuel_eui,
                                                   name="Average of Alaska " + current_building_type + " buildings")

    current_building = go.Scatter(x=current_building_df.datetime,
                                 y=current_building_df.monthly_fuel_eui,
                                 name=current_building_name)

    layout = dict(title= 'Monthly Fuel EUI comparison: ' + current_building_name + ' compared to other ' + current_building_type + ' buildings',
                  xaxis=dict(title='Date'),
                  yaxis=dict(title='Fuel energy use intensity (BTUs / square foot / month)')
                 )

    fig = go.Figure(dict(data=[all_building_avg_monthly_fuel_eui, current_building], layout=layout))
    
    if current_building_df.building_square_footage.isna().all():
        markdown_error = '''#### <font color='red'>This building has no square footage data, and so cannot be compared to other buildings.</font>'''
    else: 
        fig.show()


Markdown(markdown_error)